In [1]:
import sklearn as skl
import pandas as pd
import xgboost as xgb
import numpy as np

In [2]:
entrenamiento = pd.read_csv('Train_TP2_Datos_2020-2C.csv')
entrenamiento = entrenamiento[( entrenamiento['Stage'] == 'Closed Won') | ( entrenamiento['Stage'] == 'Closed Lost')]
test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')

In [3]:
#Agrego feature: Duracion por region
duracion_region = pd.read_csv('duracion_region.csv')
entrenamiento = entrenamiento.merge(duracion_region,on='Region',how='left')
test = test.merge(duracion_region,on='Region',how='left')

#Agrego feature: Duracion por territorio
duracion_territorio = pd.read_csv('duracion_territorio.csv')
entrenamiento = entrenamiento.merge(duracion_territorio,on='Territory',how='left')
test = test.merge(duracion_territorio,on='Territory',how='left')

#Agrego feature: Comportamiento segun territorio
comportamiento_territorio =  pd.read_csv('comportamiento_territorio.csv')
entrenamiento = entrenamiento.merge(comportamiento_territorio,on='Territory',how='left')
test = test.merge(comportamiento_territorio,on='Territory',how='left')

In [4]:
comportamiento_territorio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Territory  44 non-null     object 
 1   0/10       44 non-null     float64
 2   1/10       44 non-null     float64
 3   2/10       44 non-null     float64
 4   3/10       44 non-null     float64
 5   4/10       44 non-null     float64
 6   5/10       44 non-null     float64
 7   6/10       44 non-null     float64
 8   7/10       44 non-null     float64
 9   8/10       44 non-null     float64
 10  9/10       44 non-null     float64
 11  10/10      44 non-null     float64
dtypes: float64(11), object(1)
memory usage: 4.2+ KB


In [5]:
##ENTRENAMIENTO

#QUITO COLUMNAS INÚTILES
entrenamiento = entrenamiento.drop(columns=['ASP_(converted)_Currency','Quote_Type','Brand','Product_Type','Size','Product_Category_B','Price','Currency','Last_Activity','Actual_Delivery_Date','Prod_Category_A'])

#QUITO COLUMNAS CON DEMASIADOS VALORES
entrenamiento = entrenamiento.drop(columns=['Opportunity_Name','Sales_Contract_No'])

#TRANSFORMO A DIFERENCIA DE DIAS LAS FECHAS
def formateo(x):
    if(x == 'NaT'):
        return x
    cadena = x.split('/')
    return '{dia:02}/{mes:02}/{ano}'.format(dia=int(cadena[0]), mes=int(cadena[1]),ano=int(cadena[2]))
columnas_fecha = ['Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
for columna in columnas_fecha:
    entrenamiento[columna] = entrenamiento[columna].apply(formateo)
    entrenamiento[columna] = pd.to_datetime(entrenamiento[columna], format='%m/%d/%Y')

entrenamiento = entrenamiento.dropna()

def tiempo_en_dias(x):
    return (x - pd.to_datetime('01/01/2000', format='%m/%d/%Y')).days
for columna in columnas_fecha:
    entrenamiento[columna] = entrenamiento[columna].apply(tiempo_en_dias)
    entrenamiento[columna] = entrenamiento[columna].astype('int32')

#TRATAMIENTO ESPECIAL A MONTH
def formateo_mes(x):
    x = x.replace(" - ","/")
    cadena = x.split('/')
    return '{ano}/{mes:02}'.format(ano=int(cadena[0]), mes=int(cadena[1]))
entrenamiento['Month'] = entrenamiento['Month'].apply(formateo_mes)
entrenamiento['Month'] = pd.to_datetime(entrenamiento['Month'], format='%Y/%m')
entrenamiento['Month'] = entrenamiento['Month'].apply(tiempo_en_dias)
entrenamiento['Month'] = entrenamiento['Month'].astype('int32')

entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9965 entries, 0 to 16882
Data columns (total 52 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  9965 non-null   int64  
 1   Region                              9965 non-null   object 
 2   Territory                           9965 non-null   object 
 3   Pricing, Delivery_Terms_Quote_Appr  9965 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    9965 non-null   int64  
 5   Bureaucratic_Code_0_Approval        9965 non-null   int64  
 6   Bureaucratic_Code_0_Approved        9965 non-null   int64  
 7   Submitted_for_Approval              9965 non-null   int64  
 8   Bureaucratic_Code                   9965 non-null   object 
 9   Account_Created_Date                9965 non-null   int32  
 10  Source                              9965 non-null   object 
 11  Billing_Country                     9965 n

In [6]:
##TEST

#QUITO COLUMNAS INÚTILES
test = test.drop(columns=['ASP_(converted)_Currency','Quote_Type','Brand','Product_Type','Size','Product_Category_B','Price','Currency','Last_Activity','Actual_Delivery_Date','Prod_Category_A'])

#QUITO COLUMNAS CON DEMASIADOS VALORES
test = test.drop(columns=['Opportunity_Name','Sales_Contract_No'])

#TRANSFORMO A DIFERENCIA DE DIAS LAS FECHAS
def formateo(x):
    if(x == 'NaT'):
        return x
    cadena = x.split('/')
    return '{dia:02}/{mes:02}/{ano}'.format(dia=int(cadena[0]), mes=int(cadena[1]),ano=int(cadena[2]))
columnas_fecha = ['Last_Modified_Date','Account_Created_Date','Opportunity_Created_Date','Quote_Expiry_Date','Planned_Delivery_Start_Date','Planned_Delivery_End_Date']
for columna in columnas_fecha:
    test[columna] = test[columna].apply(formateo)
    test[columna] = pd.to_datetime(test[columna], format='%m/%d/%Y')

def tiempo_en_dias(x):
    return (x - pd.to_datetime('01/01/2000', format='%m/%d/%Y')).days
for columna in columnas_fecha:
    test[columna] = test[columna].apply(tiempo_en_dias)
    test[columna] = test[columna].astype('float')

#TRATAMIENTO ESPECIAL A MONTH
def formateo_mes(x):
    x = x.replace(" - ","/")
    cadena = x.split('/')
    return '{ano}/{mes:02}'.format(ano=int(cadena[0]), mes=int(cadena[1]))
test['Month'] = test['Month'].apply(formateo_mes)
test['Month'] = pd.to_datetime(test['Month'], format='%Y/%m')
test['Month'] = test['Month'].apply(tiempo_en_dias)
test['Month'] = test['Month'].astype('int32')

test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2551 entries, 0 to 2550
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  2551 non-null   int64  
 1   Region                              2551 non-null   object 
 2   Territory                           2551 non-null   object 
 3   Pricing, Delivery_Terms_Quote_Appr  2551 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    2551 non-null   int64  
 5   Bureaucratic_Code_0_Approval        2551 non-null   int64  
 6   Bureaucratic_Code_0_Approved        2551 non-null   int64  
 7   Submitted_for_Approval              2551 non-null   int64  
 8   Bureaucratic_Code                   2551 non-null   object 
 9   Account_Created_Date                2551 non-null   float64
 10  Source                              2551 non-null   object 
 11  Billing_Country                     2551 no

In [7]:
test['Opportunity_ID'].value_counts()

12266    26
10973    24
11089    18
11869    15
11248    11
         ..
11778     1
12181     1
12179     1
11518     1
11488     1
Name: Opportunity_ID, Length: 1567, dtype: int64

In [8]:
#QUITO STAGE
objetivo = entrenamiento.pop('Stage')
entrenamiento.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9965 entries, 0 to 16882
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  9965 non-null   int64  
 1   Region                              9965 non-null   object 
 2   Territory                           9965 non-null   object 
 3   Pricing, Delivery_Terms_Quote_Appr  9965 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    9965 non-null   int64  
 5   Bureaucratic_Code_0_Approval        9965 non-null   int64  
 6   Bureaucratic_Code_0_Approved        9965 non-null   int64  
 7   Submitted_for_Approval              9965 non-null   int64  
 8   Bureaucratic_Code                   9965 non-null   object 
 9   Account_Created_Date                9965 non-null   int32  
 10  Source                              9965 non-null   object 
 11  Billing_Country                     9965 n

In [9]:
def se_gano(x):
    return int(x == 'Closed Won')
objetivo = objetivo.apply(se_gano)
objetivo

0        0
1        1
2        1
4        0
5        0
        ..
16875    1
16876    0
16877    0
16881    0
16882    0
Name: Stage, Length: 9965, dtype: int64

In [10]:
columnas_object = list(entrenamiento.select_dtypes(include=['object']).columns)

for columna in columnas_object:
    print(columna + ":" + str(entrenamiento[columna].value_counts().size))

Region:5
Territory:44
Bureaucratic_Code:6
Source :14
Billing_Country:65
Account_Name:1320
Account_Owner:45
Opportunity_Owner:52
Account_Type:7
Opportunity_Type:25
Delivery_Terms:9
Last_Modified_By:50
Product_Family:198
Product_Name:396
ASP_Currency:5
Delivery_Quarter:4
Total_Amount_Currency:5
Total_Taxable_Amount_Currency:5


In [11]:
def categoricas_a_numericas(x):
    ohe = skl.preprocessing.OrdinalEncoder()
    for columna in columnas_object:
        copia = x[[columna]].copy().dropna()
        df_temp = pd.DataFrame(ohe.fit_transform(copia)).astype('int32')
        df_temp.columns = [columna]
        x[columna] = df_temp[columna]
        print(columna)
categoricas_a_numericas(entrenamiento)
categoricas_a_numericas(test)

Region
Territory
Bureaucratic_Code
Source 
Billing_Country
Account_Name
Account_Owner
Opportunity_Owner
Account_Type
Opportunity_Type
Delivery_Terms
Last_Modified_By
Product_Family
Product_Name
ASP_Currency
Delivery_Quarter
Total_Amount_Currency
Total_Taxable_Amount_Currency
Region
Territory
Bureaucratic_Code
Source 
Billing_Country
Account_Name
Account_Owner
Opportunity_Owner
Account_Type
Opportunity_Type
Delivery_Terms
Last_Modified_By
Product_Family
Product_Name
ASP_Currency
Delivery_Quarter
Total_Amount_Currency
Total_Taxable_Amount_Currency


In [12]:
entrenamiento.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9965 entries, 0 to 16882
Data columns (total 51 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   ID                                  9965 non-null   int64  
 1   Region                              6412 non-null   float64
 2   Territory                           6412 non-null   float64
 3   Pricing, Delivery_Terms_Quote_Appr  9965 non-null   int64  
 4   Pricing, Delivery_Terms_Approved    9965 non-null   int64  
 5   Bureaucratic_Code_0_Approval        9965 non-null   int64  
 6   Bureaucratic_Code_0_Approved        9965 non-null   int64  
 7   Submitted_for_Approval              9965 non-null   int64  
 8   Bureaucratic_Code                   6412 non-null   float64
 9   Account_Created_Date                9965 non-null   int32  
 10  Source                              6412 non-null   float64
 11  Billing_Country                     6412 n

In [13]:
d_entrenamiento = xgb.DMatrix(entrenamiento.values, objetivo.values)
d_prueba = xgb.DMatrix(test.values)

In [14]:
param = {'max_depth':5, 'eta':0.6, 'objective':'binary:logistic' }
num_round = 5
bst = xgb.train(param, d_entrenamiento, num_round)

[00:08:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [15]:
preds = bst.predict(d_prueba)

In [16]:
#pd.Series([ for x in list(preds)])
resultados = test[['Opportunity_ID']].copy()
resultados['Target'] = pd.Series(preds)
resultados = resultados.groupby('Opportunity_ID').mean()
resultados = resultados.reset_index()
def temp(x):
    return int(x >= 0.5)
resultados['Target'] = resultados['Target'].apply(temp)

In [17]:
resultados.to_csv("prediccion.csv", index=False)

In [18]:
resultados['Target'].value_counts()

1    849
0    718
Name: Target, dtype: int64

In [19]:
resultados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Opportunity_ID  1567 non-null   int64
 1   Target          1567 non-null   int64
dtypes: int64(2)
memory usage: 24.6 KB
